<a href="https://colab.research.google.com/github/MackStancil/Mack-Stancil-Portfolio/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendation System

# Content Based-Filtering

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# I had to install surprise first by using the command '!pip install scikit-surprise'
!pip install scikit-surprise
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate, KFold

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163481 sha256=52205d8e3c75f43491e4c8583ed7973699ecd2ed34b36120af7a715fc7335b80
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Open and read databases
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tmdb_5000_credits.csv")
df2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tmdb_5000_movies.csv")

# Combine database values on the 'id' column
df1.columns = ['id', 'title', 'cast', 'crew']
df_movies = df2.merge(df1, on='id')

In [ ]:
# Display 5 examples of movies from the database
df_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
# Displays the overview data of movies
df_movies['overview']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object

In [ ]:
# First use of the tfidfVectorizer import from sklearn

# Define tfidf vectorizer object. This removes english stop words like 'the' or 'a'
tfidf = TfidfVectorizer(stop_words="english")

# Replace NaN with an empty string
df_movies['overview'] = df_movies['overview'].fillna("")

# Construct the required tfidf matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df_movies['overview'])

# Output the shape of tfidf matrix
tfidf_matrix.shape

(4803, 20978)

In [ ]:
# First use of the linear_kernel import from sklearn
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
# Create a reverse map of indices and movie titles
indices = pd.Series(df_movies.index, index=df_movies['original_title']).drop_duplicates()

# Displays examples of title indices
indices

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [ ]:
# Function that takes in a movie title and outputs recommendations
def get_recommendations(title, cosine_sim=cosine_sim):
    # Gets the index of the movie
    idx = indices[title]

    # Gets the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df_movies["original_title"].iloc[movie_indices]

get_recommendations('The Dark Knight Rises')

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: original_title, dtype: object

**Credits, Genres, and Keywords**

In [ ]:
# First time using the literal_eval import from ast
# Parse the features into their corresponding python objects
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
  df_movies[feature] = df_movies[feature].apply(literal_eval)

In [ ]:
# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
  for i in x:
    if i['job'] == 'Director':
      return i['name']
  return np.nan

In [ ]:
# Returns the list top 3 elements or entire list; whichever is more
def get_list(x):
  if isinstance(x, list):
    names = [i['name'] for i in x]

    # Check if more than 3 elements exist. If yes, return only first 3. If no, return entire list
    if len(names) > 3:
      names = names[:3]
    return names

  # Return empty list in case of missing/malformed data
  return []

In [ ]:
# Define new director, cast, genres, and keywords features that are in a suitable form
df_movies['director'] = df_movies['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
  df_movies[feature] = df_movies[feature].apply(get_list)

In [ ]:
# Display example of the cleaned data
df_movies[['original_title', 'cast', 'director', 'keywords', 'genres']].head()

,original_title,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,"[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes,"[spy, based on novel, secret agent]","[Action, Adventure, Crime]"
3,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan,"[dc comics, crime fighter, terrorist]","[Action, Crime, Drama]"
4,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton,"[based on novel, mars, medallion]","[Action, Adventure, Science Fiction]"


In [ ]:
# Converts all string to lower case and removes spaces from names
def clean_data(x):
  if isinstance(x, list):
    return [str.lower(i.replace(" ", "")) for i in x]
  else:
    # Check if director exists. If not, return empty
    if isinstance(x, str):
      return str.lower(x.replace(" ", ""))
    else:
      return ''

In [ ]:
# Apply clean data to features
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
  df_movies[feature] = df_movies[feature].apply(clean_data)

In [ ]:
def create_soup(x):
  return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df_movies['soup'] = df_movies.apply(create_soup, axis=1)

In [ ]:
# First time using the CountVectorizer import from sklearn.feature_extraction.text
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df_movies['soup'])

In [ ]:
# First time using the cosine_similarity import from sklearn.metrics.pairwise
# Compute the cosine similarity matrix based on the count matrix
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# Reset index of our main dataframe and construct reverse mapping
df_movies = df_movies.reset_index()
indices = pd.Series(df_movies.index, index=df_movies['original_title'])

In [ ]:
# Example of recommendations using the new metadata
get_recommendations('The Dark Knight Rises', cosine_sim2)

65               The Dark Knight
119                Batman Begins
4638    Amidst the Devil's Wings
1196                The Prestige
3073           Romeo Is Bleeding
3326              Black November
1503                      Takers
1986                      Faster
303                     Catwoman
747               Gangster Squad
Name: original_title, dtype: object

# Collaborative FIltering

In [ ]:
# First instance of import Reader, Dataset, SVD, evaluate from surprise
reader = Reader()

# Open and read the new dataset to get user ratings
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ratings_small.csv')

# Display example of new dataset
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=5)
kf.split(data)

<generator object KFold.split at 0x7ec57d8610e0>

In [ ]:
# First instance using the cross_validate import from surprise
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'test_rmse': array([0.89343398, 0.8971497 , 0.90161061, 0.90628407, 0.8886371 ]),
 'test_mae': array([0.68849675, 0.69075423, 0.69165961, 0.69727145, 0.68492718]),
 'fit_time': (1.7610735893249512,
  1.7382006645202637,
  2.708881139755249,
  2.4463860988616943,
  1.7312202453613281),
 'test_time': (0.11985468864440918,
  0.4907529354095459,
  0.26566576957702637,
  0.12209415435791016,
  0.12703466415405273)}

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
# Displays example of a users ratings
ratings[ratings['userId']==1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [ ]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.7299315237453374, details={'was_impossible': False})

In [ ]:
# Gets the predicted rating of each movi in the database for the user and puts it in an array
pred = []
for i in range (176000):
  p = svd.predict(1, i, 3)
  pred.append(p)


In [ ]:
# Displays the top ten predicted ratings
predictions = sorted(pred, key=lambda x: x.est, reverse=True)[:10]
predictions

[Prediction(uid=1, iid=1172, r_ui=3, est=3.692365542827942, details={'was_impossible': False}),
 Prediction(uid=1, iid=296, r_ui=3, est=3.660436757445102, details={'was_impossible': False}),
 Prediction(uid=1, iid=318, r_ui=3, est=3.6158280721883225, details={'was_impossible': False}),
 Prediction(uid=1, iid=926, r_ui=3, est=3.571640051891815, details={'was_impossible': False}),
 Prediction(uid=1, iid=922, r_ui=3, est=3.5651667818744435, details={'was_impossible': False}),
 Prediction(uid=1, iid=904, r_ui=3, est=3.560279824018852, details={'was_impossible': False}),
 Prediction(uid=1, iid=3462, r_ui=3, est=3.5404572576927476, details={'was_impossible': False}),
 Prediction(uid=1, iid=1196, r_ui=3, est=3.526298946117874, details={'was_impossible': False}),
 Prediction(uid=1, iid=969, r_ui=3, est=3.514890602388939, details={'was_impossible': False}),
 Prediction(uid=1, iid=1217, r_ui=3, est=3.514415619372767, details={'was_impossible': False})]

**FINAL TESTING**

In [ ]:
get_recommendations('The Godfather')

2731     The Godfather: Part II
1873                 Blood Ties
867     The Godfather: Part III
3727                Snabba Cash
3623                       Made
3125                     Eulogy
3896                   Sinister
4506            The Maid's Room
3783                        Joe
2244      The Cold Light of Day
Name: original_title, dtype: object

Execution Time: 43ms

In [ ]:
get_recommendations('Saving Private Ryan')

787         The Great Raid
586      The Monuments Men
290      The Expendables 2
4168             Abandoned
3516             The Train
1525        Apocalypse Now
2430             Evil Dead
3603     Lone Wolf McQuade
3586    As Above, So Below
755     Star Trek: Nemesis
Name: original_title, dtype: object

Execution Time: 157ms

In [ ]:
get_recommendations('Happy Gilmore')

3638                   Rabbit Hole
1065                       Tin Cup
1066          Yours, Mine and Ours
1543                       The Vow
682                  The Love Guru
3984              Some Like It Hot
2396                   Hoodwinked!
764     The Legend of Bagger Vance
321                   The Campaign
1182           Big Momma's House 2
Name: original_title, dtype: object

Execution Time: 187ms

In [ ]:
get_recommendations('Up')

2786    I Love You Phillip Morris
962                 Raising Helen
1090        All the Pretty Horses
2799           Dancer in the Dark
3313                 An Education
1122             Along Came Polly
3134                    Partition
3137                    Nurse 3-D
3660        Farce of the Penguins
2246         Synecdoche, New York
Name: original_title, dtype: object

Execution Time: 142ms

In [ ]:
get_recommendations('Iron Man')

79                     Iron Man 2
31                     Iron Man 3
1868           Cradle 2 the Grave
7         Avengers: Age of Ultron
538                       Hostage
119                 Batman Begins
1290     Baahubali: The Beginning
4574                     Roadside
2044           The Little Vampire
2633    The Clan of the Cave Bear
Name: original_title, dtype: object

Execution Time: 174ms

In [ ]:
get_recommendations('Jurassic Park')

28                   Jurassic World
2527    National Lampoon's Vacation
1580                    The Nut Job
479          Walking With Dinosaurs
2805           The Land Before Time
1983               Meet the Deedles
3577               The Way Way Back
54                The Good Dinosaur
3139                  Adventureland
1536                       Vacation
Name: original_title, dtype: object

Execution Time: 303ms

In [ ]:
get_recommendations('Finding Nemo')

3124                                             The Reef
3568                         20,000 Leagues Under the Sea
3389                                Chairman of the Board
2069                     Teenage Mutant Ninja Turtles III
2233                                           Little Boy
525                                            Shark Tale
1827    Teenage Mutant Ninja Turtles II: The Secret of...
326                                            Cinderella
4635    The Incredibly True Adventure of Two Girls In ...
2742                                         The Shallows
Name: original_title, dtype: object

Execution Time: 161ms

In [ ]:
get_recommendations('Scream')

4498                             The Kentucky Fried Movie
2842                                       Stir of Echoes
3988                                    Diary of the Dead
3659                       Dead Like Me: Life After Death
4159                                         River's Edge
4053                     Friday the 13th: A New Beginning
2762                                           Serial Mom
3751                         What the #$*! Do We (K)now!?
1723    Alexander and the Terrible, Horrible, No Good,...
8                  Harry Potter and the Half-Blood Prince
Name: original_title, dtype: object

Execution Time: 145ms

In [ ]:
get_recommendations('Superbad')

4467     American Graffiti
2810          American Pie
2828             Project X
2227    Battle of the Year
2036      Saving Silverman
3255            Half Baked
2421                   ATL
3048       Very Bad Things
2880      Charlie Bartlett
3481            Miss March
Name: original_title, dtype: object

Execution Time: 147ms

In [ ]:
get_recommendations('Se7en')

1379              The Cell
1009    The Bone Collector
968      Murder by Numbers
761         Righteous Kill
1905       My Soul to Take
2138               Copycat
3853                  2:13
421                 Zodiac
1020               Taken 3
1320            Switchback
Name: original_title, dtype: object

Execution Time: 0.152 seconds

Average Execution Time: 199ms

In [ ]:
pred = []
for i in range (176000):
  p = svd.predict(2, i, 3)
  pred.append(p)

predictions = sorted(pred, key=lambda x: x.est, reverse=True)[:10]
predictions

[Prediction(uid=2, iid=922, r_ui=3, est=4.393438647370116, details={'was_impossible': False}),
 Prediction(uid=2, iid=318, r_ui=3, est=4.390037527708509, details={'was_impossible': False}),
 Prediction(uid=2, iid=969, r_ui=3, est=4.376666846943714, details={'was_impossible': False}),
 Prediction(uid=2, iid=56782, r_ui=3, est=4.365568318990554, details={'was_impossible': False}),
 Prediction(uid=2, iid=1221, r_ui=3, est=4.323067077817439, details={'was_impossible': False}),
 Prediction(uid=2, iid=1945, r_ui=3, est=4.319846812387588, details={'was_impossible': False}),
 Prediction(uid=2, iid=1172, r_ui=3, est=4.310787236753048, details={'was_impossible': False}),
 Prediction(uid=2, iid=8132, r_ui=3, est=4.306715459426233, details={'was_impossible': False}),
 Prediction(uid=2, iid=7502, r_ui=3, est=4.303511804108258, details={'was_impossible': False}),
 Prediction(uid=2, iid=899, r_ui=3, est=4.297279521943983, details={'was_impossible': False})]

Execution Time: 2622ms

In [ ]:
pred = []
for i in range (176000):
  p = svd.predict(3, i, 3)
  pred.append(p)

predictions = sorted(pred, key=lambda x: x.est, reverse=True)[:10]
predictions

[Prediction(uid=3, iid=111, r_ui=3, est=4.306141795813716, details={'was_impossible': False}),
 Prediction(uid=3, iid=318, r_ui=3, est=4.2904437218415294, details={'was_impossible': False}),
 Prediction(uid=3, iid=1197, r_ui=3, est=4.221568702639014, details={'was_impossible': False}),
 Prediction(uid=3, iid=2019, r_ui=3, est=4.214829294540374, details={'was_impossible': False}),
 Prediction(uid=3, iid=475, r_ui=3, est=4.1907932660866445, details={'was_impossible': False}),
 Prediction(uid=3, iid=56782, r_ui=3, est=4.19050384129671, details={'was_impossible': False}),
 Prediction(uid=3, iid=2542, r_ui=3, est=4.1878534443086926, details={'was_impossible': False}),
 Prediction(uid=3, iid=1228, r_ui=3, est=4.1811729319944995, details={'was_impossible': False}),
 Prediction(uid=3, iid=1221, r_ui=3, est=4.176628073491776, details={'was_impossible': False}),
 Prediction(uid=3, iid=904, r_ui=3, est=4.1669548464929935, details={'was_impossible': False})]

Execution Time: 2032ms

In [ ]:
pred = []
for i in range (176000):
  p = svd.predict(5, i, 3)
  pred.append(p)

predictions = sorted(pred, key=lambda x: x.est, reverse=True)[:10]
predictions

[Prediction(uid=5, iid=318, r_ui=3, est=4.799980890336844, details={'was_impossible': False}),
 Prediction(uid=5, iid=922, r_ui=3, est=4.797520433011796, details={'was_impossible': False}),
 Prediction(uid=5, iid=1228, r_ui=3, est=4.701539199490448, details={'was_impossible': False}),
 Prediction(uid=5, iid=1259, r_ui=3, est=4.680421225690238, details={'was_impossible': False}),
 Prediction(uid=5, iid=1172, r_ui=3, est=4.674838241977547, details={'was_impossible': False}),
 Prediction(uid=5, iid=1276, r_ui=3, est=4.672909945002504, details={'was_impossible': False}),
 Prediction(uid=5, iid=969, r_ui=3, est=4.667641116685157, details={'was_impossible': False}),
 Prediction(uid=5, iid=50, r_ui=3, est=4.627739158808931, details={'was_impossible': False}),
 Prediction(uid=5, iid=953, r_ui=3, est=4.619881654081704, details={'was_impossible': False}),
 Prediction(uid=5, iid=1217, r_ui=3, est=4.60673325122114, details={'was_impossible': False})]

Execution Time: 1769ms

In [ ]:
pred = []
for i in range (176000):
  p = svd.predict(6, i, 3)
  pred.append(p)

predictions = sorted(pred, key=lambda x: x.est, reverse=True)[:10]
predictions

[Prediction(uid=6, iid=2318, r_ui=3, est=4.25259869790233, details={'was_impossible': False}),
 Prediction(uid=6, iid=296, r_ui=3, est=4.192185036014074, details={'was_impossible': False}),
 Prediction(uid=6, iid=905, r_ui=3, est=4.186476661819521, details={'was_impossible': False}),
 Prediction(uid=6, iid=1252, r_ui=3, est=4.182757775353913, details={'was_impossible': False}),
 Prediction(uid=6, iid=1204, r_ui=3, est=4.166569309360663, details={'was_impossible': False}),
 Prediction(uid=6, iid=1203, r_ui=3, est=4.155298259259125, details={'was_impossible': False}),
 Prediction(uid=6, iid=608, r_ui=3, est=4.141519037849083, details={'was_impossible': False}),
 Prediction(uid=6, iid=7502, r_ui=3, est=4.1154295983731135, details={'was_impossible': False}),
 Prediction(uid=6, iid=912, r_ui=3, est=4.109305618705819, details={'was_impossible': False}),
 Prediction(uid=6, iid=4235, r_ui=3, est=4.102846082305691, details={'was_impossible': False})]

Execution Time: 1248ms

In [ ]:
pred = []
for i in range (176000):
  p = svd.predict(7, i, 3)
  pred.append(p)

predictions = sorted(pred, key=lambda x: x.est, reverse=True)[:10]
predictions

[Prediction(uid=7, iid=858, r_ui=3, est=4.586687945244511, details={'was_impossible': False}),
 Prediction(uid=7, iid=593, r_ui=3, est=4.490218675047865, details={'was_impossible': False}),
 Prediction(uid=7, iid=926, r_ui=3, est=4.480904025693684, details={'was_impossible': False}),
 Prediction(uid=7, iid=318, r_ui=3, est=4.46570863586109, details={'was_impossible': False}),
 Prediction(uid=7, iid=1221, r_ui=3, est=4.409305025482952, details={'was_impossible': False}),
 Prediction(uid=7, iid=527, r_ui=3, est=4.407924993867668, details={'was_impossible': False}),
 Prediction(uid=7, iid=1203, r_ui=3, est=4.395679910148633, details={'was_impossible': False}),
 Prediction(uid=7, iid=50, r_ui=3, est=4.370707045611856, details={'was_impossible': False}),
 Prediction(uid=7, iid=905, r_ui=3, est=4.339899723120557, details={'was_impossible': False}),
 Prediction(uid=7, iid=898, r_ui=3, est=4.306147105419046, details={'was_impossible': False})]

Execution Time: 2072ms

In [ ]:
pred = []
for i in range (176000):
  p = svd.predict(8, i, 3)
  pred.append(p)

predictions = sorted(pred, key=lambda x: x.est, reverse=True)[:10]
predictions

[Prediction(uid=8, iid=318, r_ui=3, est=4.637051304889779, details={'was_impossible': False}),
 Prediction(uid=8, iid=969, r_ui=3, est=4.622292927445891, details={'was_impossible': False}),
 Prediction(uid=8, iid=4878, r_ui=3, est=4.57990008738446, details={'was_impossible': False}),
 Prediction(uid=8, iid=858, r_ui=3, est=4.578762915033815, details={'was_impossible': False}),
 Prediction(uid=8, iid=2019, r_ui=3, est=4.50966334296104, details={'was_impossible': False}),
 Prediction(uid=8, iid=1276, r_ui=3, est=4.498808369407589, details={'was_impossible': False}),
 Prediction(uid=8, iid=2329, r_ui=3, est=4.471238854636477, details={'was_impossible': False}),
 Prediction(uid=8, iid=1945, r_ui=3, est=4.44985168442959, details={'was_impossible': False}),
 Prediction(uid=8, iid=898, r_ui=3, est=4.4490196819420085, details={'was_impossible': False}),
 Prediction(uid=8, iid=3462, r_ui=3, est=4.43607124649446, details={'was_impossible': False})]

Execution Time: 1568ms

In [ ]:
pred = []
for i in range (176000):
  p = svd.predict(9, i, 3)
  pred.append(p)

predictions = sorted(pred, key=lambda x: x.est, reverse=True)[:10]
predictions

[Prediction(uid=9, iid=527, r_ui=3, est=4.475807115291094, details={'was_impossible': False}),
 Prediction(uid=9, iid=858, r_ui=3, est=4.473062542743233, details={'was_impossible': False}),
 Prediction(uid=9, iid=8132, r_ui=3, est=4.431684758943948, details={'was_impossible': False}),
 Prediction(uid=9, iid=1204, r_ui=3, est=4.416436627384373, details={'was_impossible': False}),
 Prediction(uid=9, iid=50, r_ui=3, est=4.379363155701672, details={'was_impossible': False}),
 Prediction(uid=9, iid=3468, r_ui=3, est=4.36709756875997, details={'was_impossible': False}),
 Prediction(uid=9, iid=926, r_ui=3, est=4.344217525621575, details={'was_impossible': False}),
 Prediction(uid=9, iid=923, r_ui=3, est=4.340923261863101, details={'was_impossible': False}),
 Prediction(uid=9, iid=36, r_ui=3, est=4.34015583969469, details={'was_impossible': False}),
 Prediction(uid=9, iid=1228, r_ui=3, est=4.33519396179163, details={'was_impossible': False})]

Execution Time: 1413ms

In [ ]:
pred = []
for i in range (176000):
  p = svd.predict(10, i, 3)
  pred.append(p)

predictions = sorted(pred, key=lambda x: x.est, reverse=True)[:10]
predictions

[Prediction(uid=10, iid=50, r_ui=3, est=4.6179296934524094, details={'was_impossible': False}),
 Prediction(uid=10, iid=858, r_ui=3, est=4.594878944734599, details={'was_impossible': False}),
 Prediction(uid=10, iid=969, r_ui=3, est=4.560156189243953, details={'was_impossible': False}),
 Prediction(uid=10, iid=926, r_ui=3, est=4.504432270982565, details={'was_impossible': False}),
 Prediction(uid=10, iid=913, r_ui=3, est=4.485650886438073, details={'was_impossible': False}),
 Prediction(uid=10, iid=1221, r_ui=3, est=4.471030441375386, details={'was_impossible': False}),
 Prediction(uid=10, iid=58559, r_ui=3, est=4.463090739801612, details={'was_impossible': False}),
 Prediction(uid=10, iid=908, r_ui=3, est=4.453795355551512, details={'was_impossible': False}),
 Prediction(uid=10, iid=70286, r_ui=3, est=4.453263834487705, details={'was_impossible': False}),
 Prediction(uid=10, iid=905, r_ui=3, est=4.445078963430062, details={'was_impossible': False})]

Execution Time: 2313ms



In [ ]:
pred = []
for i in range (176000):
  p = svd.predict(11, i, 3)
  pred.append(p)

predictions = sorted(pred, key=lambda x: x.est, reverse=True)[:10]
predictions

[Prediction(uid=11, iid=318, r_ui=3, est=4.9091326023180075, details={'was_impossible': False}),
 Prediction(uid=11, iid=905, r_ui=3, est=4.871845646765208, details={'was_impossible': False}),
 Prediction(uid=11, iid=926, r_ui=3, est=4.866337122495146, details={'was_impossible': False}),
 Prediction(uid=11, iid=4973, r_ui=3, est=4.83927242509831, details={'was_impossible': False}),
 Prediction(uid=11, iid=969, r_ui=3, est=4.820313773668323, details={'was_impossible': False}),
 Prediction(uid=11, iid=50, r_ui=3, est=4.790672271638662, details={'was_impossible': False}),
 Prediction(uid=11, iid=3462, r_ui=3, est=4.786370149809349, details={'was_impossible': False}),
 Prediction(uid=11, iid=1197, r_ui=3, est=4.78222107387378, details={'was_impossible': False}),
 Prediction(uid=11, iid=2132, r_ui=3, est=4.778596985405472, details={'was_impossible': False}),
 Prediction(uid=11, iid=922, r_ui=3, est=4.776048835744364, details={'was_impossible': False})]

Execution Time: 3245ms

In [ ]:
pred = []
for i in range (176000):
  p = svd.predict(12, i, 3)
  pred.append(p)

predictions = sorted(pred, key=lambda x: x.est, reverse=True)[:10]
predictions

[Prediction(uid=12, iid=2804, r_ui=3, est=3.9001649002450747, details={'was_impossible': False}),
 Prediction(uid=12, iid=904, r_ui=3, est=3.8931270546340695, details={'was_impossible': False}),
 Prediction(uid=12, iid=1172, r_ui=3, est=3.8389198996989973, details={'was_impossible': False}),
 Prediction(uid=12, iid=915, r_ui=3, est=3.8087874179148336, details={'was_impossible': False}),
 Prediction(uid=12, iid=1204, r_ui=3, est=3.80837442275055, details={'was_impossible': False}),
 Prediction(uid=12, iid=2917, r_ui=3, est=3.807564349671726, details={'was_impossible': False}),
 Prediction(uid=12, iid=2318, r_ui=3, est=3.8060715202052053, details={'was_impossible': False}),
 Prediction(uid=12, iid=6016, r_ui=3, est=3.773082393039723, details={'was_impossible': False}),
 Prediction(uid=12, iid=1217, r_ui=3, est=3.7501108764216453, details={'was_impossible': False}),
 Prediction(uid=12, iid=1221, r_ui=3, est=3.7442750086904932, details={'was_impossible': False})]

Execution Time: 1962ms

Average Execution Time: 2024ms